# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll then also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

## Loading the Data

Start by loading in the data stored in the file **ToothGrowth.csv**.

In [2]:
# Your code here
import pandas as pd

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generating the ANOVA Table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage 

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
#Your code here
f = 'len ~ C(supp) + dose'
lm = ols(f, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)


               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Reading the Table

Make a brief comment regarding the statistics regarding the effect of supplement and dosage on tooth length.

#Your comment here

Both does and supplement type are impactful. At first glance, dosage seems to be the more impactful of the two.

## Comparing to T-Tests

Now that you've gotten a brief chance to interact with ANOVA, its interesting to compare the results to those from the t-tests you were just working with. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterwards, you'll conduct a t-test to compare the tooth length of these two different samples.

In [3]:
#Your code here
data_oj_length = data[data['supp'] == 'OJ']['len']
data_vc_length = data[data['supp'] == 'VC']['len']

Now compare a t-test between these two groups and print the associated two-sided p-value.

In [4]:
import scipy.stats as stats

def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """
    numerator = a.mean() - b.mean()
    denominator = (a.var(ddof=1) / len(a) + b.var(ddof=1) / len(b))**.5

    
    return abs(numerator / denominator) #Return the t-score!

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    numerator = (a.var(ddof=1)/len(a) + b.var(ddof=1)/len(b))**2
    denominator = a.var(ddof=1)**2 / (len(a)**2 * (len(a) - 1)) + b.var(ddof=1)**2 / (len(b)**2 * (len(b) - 1))
    return numerator / denominator #Return the degrees of freedom

def p_value_welch_ttest(a, b, two_sided=False):
    #Your code here
    
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    one_tail_p_value = 1 - stats.t.cdf(t, df)
    
    return one_tail_p_value if not two_sided else one_tail_p_value * 2 #Return the p-value!

In [5]:
#Your code here; calculate the 2-sided p-value for a t-test comparing the two supplement groups.
p_value_welch_ttest(data_oj_length, data_vc_length, two_sided=True)


0.06063450788093383

## A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [6]:
#Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)


#Your code here
f = 'len ~ C(supp)'
lm = ols(f, data).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)




               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Generating Multiple T-Tests

While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [9]:
for group in data.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2 sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [10]:
#Your code here; reuse your $t$-test code above to calculate the p-value for a 2-sided $t$-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)

for group in data.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

KeyError: 'supp'

## Summary

In this lesson, you examined the ANOVA technique to generalize A/B testing methods to multiple groups and factors.